# **2388. Change Null Values in a Table to the Previous Value**

**Table: CoffeeShop**
``` console
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| id          | int     |
| drink       | varchar |
+-------------+---------+
``` 
id is the primary key (column with unique values) for this table.

Each row in this table shows the order id and the name of the drink ordered. Some drink rows are nulls.
 

Write a solution to replace the null values of the drink with the name of the drink of the previous row that is not null. It is guaranteed that the drink on the first row of the table is not null.

Return the result table in the same order as the input.

The result format is shown in the following example.

 

**Example 1:**

**Input:** 
**CoffeeShop table:**
``` console
+----+-------------------+
| id | drink             |
+----+-------------------+
| 9  | Rum and Coke      |
| 6  | null              |
| 7  | null              |
| 3  | St Germain Spritz |
| 1  | Orange Margarita  |
| 2  | null              |
+----+-------------------+
```
**Output:** 
``` console
+----+-------------------+
| id | drink             |
+----+-------------------+
| 9  | Rum and Coke      |
| 6  | Rum and Coke      |
| 7  | Rum and Coke      |
| 3  | St Germain Spritz |
| 1  | Orange Margarita  |
| 2  | Orange Margarita  |
+----+-------------------+
```
**Explanation:** 
- For ID 6, the previous value that is not null is from ID 9. We replace the null with "Rum and Coke".
- For ID 7, the previous value that is not null is from ID 9. We replace the null with "Rum and Coke;.
- For ID 2, the previous value that is not null is from ID 1. We replace the null with "Orange Margarita".
- Note that the rows in the output are the same as in the input.

**Solution-1:**

``` sql
with cte as(
select row_number() over() as rn,
id, drink
from coffeeshop
)
select a.id, coalesce(a.drink, b.drink) as drink
from cte a
LEFT JOIN cte b
on a.rn > b.rn and b.drink is not null 
group by a.id
order by a.rn, a.id

**Solution-2:**

``` sql
WITH cte AS (SELECT id, drink, ROW_NUMBER() OVER () AS nb FROM CoffeeShop), -- nb = initial row order
     cte2 AS (SELECT id, drink, nb, SUM(1-ISNULL(drink)) OVER (ORDER BY nb) AS group_id FROM cte) -- same group_id -> same drink
SELECT id, FIRST_VALUE(drink) OVER (PARTITION BY group_id) AS drink
FROM cte2
ORDER BY nb;

**Solution-3:**

``` sql
with cte as (
    select id, drink, row_number() over() as rn
    from CoffeeShop
),

cte2 as (
    select id, drink, rn, sum(if(drink is NULL, 0, 1)) over (order by rn) as group_id
    from cte
)

select id, first_value(drink) over (partition by group_id order by rn) as drink
from cte2

**Solution-4:**

``` sql
WITH cte AS (
    SELECT
        *,
        SUM(IF(drink IS NULL, 0,1 )) over (ROWS BETWEEN UNBOUNDED preceding AND CURRENT ROW) AS grp
    ,ROW_NUMBER() OVER(ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as rn
    FROM
        CoffeeShop 
)

SELECT
    id,
    first_value(drink) over(PARTITION by grp order by rn) AS drink
FROM
    cte;

**Solution-5:**

``` sql
WITH cte AS (
    SELECT
        *,
        SUM(IF(drink IS not NULL, 1,0 )) over (ROWS BETWEEN UNBOUNDED preceding AND CURRENT ROW) AS grp
    ,ROW_NUMBER() OVER(ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as rn
    FROM
        CoffeeShop 
)

SELECT
    id,
    first_value(drink) over(PARTITION by grp order by rn) AS drink
FROM
    cte;

**Solution-6:**

``` sql
WITH cte AS 
(SELECT *, SUM(IF(drink IS NOT NULL,1,0)) OVER(rows between unbounded preceding and current row) AS grp --#Refer Note 1
 FROM coffeeshop)
 
 SELECT id, MAX(drink) OVER(PARTITION BY grp) AS drink
 FROM cte;
 
 -- Note1: When using windows function without order by clause , the range of rows has to be mentioned to find cumulative sum. By default, it calculates total sum for rows in RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING